# Google Map Temporal Distances AI Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, PReLU
import pandas as pd

### Load dataset

In [ ]:
trips_original = pd.read_json("datasets/trips_france.json")
display(trips_original.head(10))

### Split into training and testing

**Nota Bene:** We will make the assumption that a trip takes the same duration in both directions (ie that they are symetric). This way we can double the size of our dataset essentially for free. We will also consider that the time it take to go from city A to itself is 0.

In [ ]:
# duplicate the trips dataframe
trips = trips_original.copy(deep=True)

# add symetry
trips_2 = trips.copy(deep=True)
trips_2["start_city_name"], trips_2["start_city_lat"], trips_2["start_city_lng"] = trips["end_city_name"], trips["end_city_lat"], trips["end_city_lng"]
trips_2["end_city_name"], trips_2["end_city_lat"], trips_2["end_city_lng"] = trips["start_city_name"], trips["start_city_lat"], trips["start_city_lng"]
trips = pd.concat([trips, trips_2])

# add zeros
zeros = trips.copy(deep=True)
zeros["end_city_name"] = zeros["start_city_name"]
zeros["end_city_lat"] = zeros["start_city_lat"]
zeros["end_city_lng"] = zeros["start_city_lng"]
zeros["duration"] = 0
trips = pd.concat([trips, zeros])

# # shuffle
# trips = trips.sample(frac = 1)

# # split input and output
# train_input = trips[["start_city_lat", "start_city_lng", "end_city_lat", "end_city_lng"]]
# train_output = trips[["duration"]]

# # show dataset's size
# print(len(trips))

### Create the model
The use a PReLU for the activation function as it seems to be really good at preventing vanishing gradient problems.

In [ ]:
model = Sequential(
  [
    Input(shape=(4,), name="layer_input"),
    Dense(10, activation=PReLU(), name="layer_1"),
    Dense(20, activation=PReLU(), name="layer_2"),
    Dense(20, activation=PReLU(), name="layer_3"),
    Dense(20, activation=PReLU(), name="layer_4"),
    Dense(10, activation=PReLU(), name="layer_5"),
    Dense(1, activation=PReLU(), name="layer_output"),
  ]
)

model.compile(optimizer="adam", loss="mse")
model.build()
model.summary()

### Train the model

In [ ]:
history = model.fit(train_input, train_output, batch_size=100, epochs=50, validation_data=(train_input, train_output))

## Backup the trained model

In [ ]:
# TODO